<a href="https://colab.research.google.com/github/qianzhou1982/Demo/blob/master/Tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tests

## Mise en place de l'environnement de travail

- Cliquez sur l'icone dossier du panneau latéral, puis connectez votre Drive
- Exécutez la commande suivante, qui va récupérer le matériel requis et le mettre dans le dossier `python-avance/tests/` de votre Drive, pour que vous ne perdiez pas votre travail. Cette commande va aussi changer le répertoire courant du notebook pour pointer vers ce dossier.

In [ ]:
!pip install pytest mutmut hypothesis deal

import pathlib

drive_dir = pathlib.Path("/content") / "drive" / "MyDrive"


if not drive_dir.exists():
  print("Vous n'avez pas monté votre Drive, consultez le texte ci-dessus.")
else:
  root_dir = drive_dir / "python-avance"
  root_dir.mkdir(parents=True, exist_ok=True)
  tests_dir = root_dir / "tests"

  if not tests_dir.exists():
    !git clone https://github.com/nzmognzmp/tp-tests.git {str(tests_dir)}

  if pathlib.Path.cwd() != tests_dir:
    %cd {tests_dir}

## Mise en place d'une démarche TDD

Voici le cahier des charges d'un système de gestion de dépendances que vous devez livrer à un de vos clients :

Pour une application de location de voitures, vous devez proposer un formulaire de sélection d'options. Les options peuvent avoir des dépendances entre elles (le système GPS ne peut être sélectionné que si l'USB l'est aussi par exemple). Cela veut dire pour le formulaire que si l'utilisateur désactive l'USB, le GPS doit être désactivé automatiquement.

Il est aussi possible que certaines options soient incompatibles : par exemple, si vous souhaitez sélectionner un lecteur de musique alors que le GPS est déjà sélectionné, et comme les deux utilisent le seul port USB disponible, la sélection ne doit pas être possible.

Formalisation préliminaire :

Ensemble de règles

- On dit que l'option A dépend de l'opion B si pour que A soit sélectionnée, B doit l'être aussi

- On dit que l'option A est en conflit avec l'option B, si pour que A soit sélectionnée, B doit être désélectionnée et inversement.

On dit qu'un ensemble de dépendances et de conflits est cohérent si il ne génère pas de contradiction (une contradiction serait par exemple, pour que A soit sélectionnée, qu'il faille que B soit sélectionné et désélectionné).

Par exemple, cet ensemble de règles serait cohérent, avec les dépendances notées `A -> B` pour `A` dépend de `B` et les conflits notés `A | B` :

```
A -> B
B -> C
C | D
```

Celui-ci ne le serait pas :

```python
A -> B
A | B
```

Mettez en place une démarche TDD pour créer ce système de règles. Pour l'instant, intéressez-vous seulement à la modélisation des dépendances et des conflits ainsi qu'au calcul de la cohérence (des règles sont-elles cohérentes ?). On regardera la sélection à proprement parler plus tard.

Pour être compatible avec la suite du TP, il est conseillé de nommer votre classe `RuleSet` d'ajouter les méthodes `add_dep`, `add_conflict` et `is_coherent`.

In [ ]:
# Votre code ici

### Solution

In [ ]:
from collections import defaultdict


class RuleSet:
  def __init__(self):
    self.deps = defaultdict(set)
    self.conflicts = defaultdict(set)

  def add_dep(self, a, b):
    self.deps[a].add(b)

  def add_conflict(self, a, b):
    self.conflicts[a].add(b)
    self.conflicts[b].add(a)

  def is_coherent(self):
    options = frozenset(self.deps)
    for option in options:
      deps = self.compute_deps(option)
      for dep in deps:
        if self.conflicts[dep] & deps:
          return False
    return True

  def compute_deps(self, option):
    to_visit = {option}
    deps = set()
    while to_visit:
      dep = to_visit.pop()
      deps.add(dep)
      to_visit.update(self.deps.get(dep, set()) - deps)
    return deps


def test_depends_aa():
  rs = RuleSet()

  rs.add_dep("a", "a")

  assert rs.is_coherent()


test_depends_aa()


def test_depends_ab_ba():
  rs = RuleSet()

  rs.add_dep("a", "b")
  rs.add_dep("b", "a")

  assert rs.is_coherent()


test_depends_ab_ba()


def test_exclusive_ab():
  rs = RuleSet()

  rs.add_dep("a", "b")
  rs.add_conflict("a", "b")

  assert not rs.is_coherent()


test_exclusive_ab()



def test_exclusive_ab_bc():
  rs = RuleSet()

  rs.add_dep("a", "b")
  rs.add_dep("b", "c")
  rs.add_conflict("a", "c")

  assert not rs.is_coherent()


test_exclusive_ab_bc()


def test_deep_deps():
  rs = RuleSet()

  rs.add_dep("a", "b")
  rs.add_dep("b", "c")
  rs.add_dep("c", "d")
  rs.add_dep("d", "e")
  rs.add_dep("a", "f")
  rs.add_conflict("e", "f")

  assert not rs.is_coherent()


test_deep_deps()

## Utilisation de `pytest`

Créez un fichier `ruleset.py` et un fichier `ruleset_tests.py` dans le dossier `/content/drive/MyDrive/python-avance/tests` et copiez-y votre code, en séparant correctement entre code applicatif et code de test.

Utilisez maintenant `pytest` pour exécuter vos tests.

In [ ]:
%%bash
# Votre utilisation de pytest ici

### Solution

In [ ]:
from pathlib import Path


if not Path("ruleset.py").exists():
  !wget https://raw.githubusercontent.com/nzmognzmp/tp-tests/correction/ruleset.py
if not Path("ruleset_tests.py").exists():
  !wget https://raw.githubusercontent.com/nzmognzmp/tp-tests/correction/ruleset_tests.py
!pytest ruleset_tests.py

## Utilisation avancée de `pytest`

Utilisez une [fixture](https://docs.pytest.org/en/stable/fixture.html) pour créer un `RuleSet` que vous utiliserez pour plusieurs tests.

Utilisez [`parametrize`](https://docs.pytest.org/en/stable/parametrize.html#pytest-mark-parametrize-parametrizing-test-functions) pour factoriser votre code de test.

In [ ]:
%%bash
# Votre utilisation avancée de pytest ici

### Solution

In [ ]:
if not Path("ruleset_adv_tests.py").exists():
  !wget https://raw.githubusercontent.com/nzmognzmp/tp-tests/correction/ruleset_adv_tests.py

!pytest ruleset_adv_tests.py

## Tests par mutations

En ayant défini une base de tests `pytest`, il est déjà possible d'utiliser certaines librairies avancées comme [`mutmut`](https://mutmut.readthedocs.io/en/latest/), qui sont basées sur `pytest`.

Testez votre code avec un test par mutations en utilisant `mutmut`

In [ ]:
%%bash
# Votre code ici

### Solution

In [ ]:
!mutmut run --paths-to-mutate ruleset.py --runner "pytest ruleset_tests.py"

In [ ]:
!mutmut show 10

## Tests par génération

D'autres librairies se concentrent sur la génération de cas. C'est le cas de l'excellente librairie `hypothesis`.

En utilisant des stratégies appropriées, testez votre `RuleSet` avec des exemples générés aléatoirement.

In [ ]:
# Votre code ici

### Solution

In [ ]:
from hypothesis import given
from hypothesis.strategies import integers, sets, tuples


@given(sets(tuples(integers(0, 5), integers(0, 5))))
def test_undepended_conflict(deps):
  rs = RuleSet()
  for a, b in deps:
    rs.add_dep(a, b)
  for i in range(6):
    rs.add_conflict(i, 6)
  assert rs.is_coherent()

test_undepended_conflict()

## Test de code existant

Dans la première cellule de ces travaux pratiques, vous avez récupéré un fichier `options.py`, qui contient l'implémentation du mécanisme de sélection.

Écrivez des tests compatibles avec `pytest` et `mutmut` pour vous assurer que ce code répond aux exigences de l'énoncé.

Visez l'absence de mutants survivants.

In [ ]:
# Votre code ici

### Solution

In [ ]:
if not Path("options_tests.py").exists():
  !wget https://raw.githubusercontent.com/nzmognzmp/tp-tests/correction/options_tests.py
!mutmut run --paths-to-mutate options.py --runner "pytest options_tests.py"

## Contrats

La programmation par contrats est très intéressantes pour le cœur du logiciel, qui gère la logique métier.

Agrémentez la classe `RuleSet` de contrats pour vérifier les hypothèses que vous faites sur sa logique. Pour cela, vous pourrez utiliser l'excellente librairie [`deal`](https://deal.readthedocs.io/index.html).

Vous pouvez la recopier ci-dessous et la tester de manière ad-hoc dans la cellule.

In [ ]:
# Votre code ici

### Solution

In [ ]:
from collections import defaultdict

import deal


@deal.inv(lambda rs: (not hasattr(rs, "conflicts")
                      or all(k not in v for k, v in rs.conflicts.items())))
class RuleSet:
    def __init__(self):
        self.deps = defaultdict(set)
        self.conflicts = defaultdict(set)

    def add_dep(self, a, b):
        self.deps[a].add(b)

    @deal.pre(lambda s, a, b: a != b)
    def add_conflict(self, a, b):
        self.conflicts[a].add(b)
        self.conflicts[b].add(a)

    def is_coherent(self):
        options = frozenset(self.deps)
        for option in options:
            deps = self.compute_deps(option)
            for dep in deps:
                if self.conflicts[dep] & deps:
                    return False
        return True

    def compute_deps(self, option):
        to_visit = {option}
        deps = set()
        while to_visit:
            dep = to_visit.pop()
            deps.add(dep)
            to_visit.update(self.deps.get(dep, set()) - deps)
        return deps

In [ ]:
rs = RuleSet()
rs.add_conflict("a", "b")

In [ ]:
rs.add_conflict("a", "a")